# 오차역전파법 (backpropagation)

덧셈 노드의 역전파 : 상류의 값을 그대로 흘린다

곱셈 노드의 역전파 : 입력신호가 서로 바뀐채로 흐른다 (순전파시에 입력 신호를 변수에 저장해야함)

#### 곱셈 계층 구현

In [11]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x  #순전파시에 입력신호 저장
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):  #dout : 상류에서 넘어온 미분
        dx = dout * self.y  #곱셈이기 때문에 x와 y가 바뀜
        dy = dout * self.x
        
        return dx,dy

In [12]:
#순전파 구현
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_apple_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [13]:
#역전파 구현
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

#### 덧셈 계층 구현

In [14]:
class AddLayer:
    def __init__(self):
        pass  #초기화가 필요없기 때문에 아무것도 하지말라 라는 뜻
    
    def forward(self,x,y):
        out = x+y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [15]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_price = MulLayer()
mul_orange_price = MulLayer()
add_tot_price = AddLayer()
mul_tot_tax_price = MulLayer()

apple_price = mul_apple_price.forward(apple,apple_num)
orange_price = mul_orange_price.forward(orange, orange_num)
tot_price = add_tot_price.forward(apple_price,orange_price)
tot_tax_price = mul_tot_tax_price.forward(tot_price,tax)

print(tot_tax_price)

dtot_tax_price = 1
dtot_price , dtax = mul_tot_tax_price.backward(dtot_tax_price)
dapple_price, dorange_price = add_tot_price.backward(dtot_price)
dapple, dapple_num = mul_apple_price.backward(dapple_price)
dorange, dorange_num = mul_orange_price.backward(dorange_price)

print(dapple, dapple_num, dorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


## 활성화 함수 계층 구현

#### RELU함수 구현

In [16]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)  #x가 0보다 크면 false, 작으면 true를 반환한다.
        out = x.copy()  #x를 얕은 copy를 통해 out에 집어넣는다. x를 바꿔도 out은 그대로 유지된다
        out[self.mask] = 0  #mask가 true인 부분(x가 0보다 작은 부분)에 0을 반환한다. out[mask]=0은 mask가 true 값을 갖는 인덱스의 넘파이 값이 모두 0이 되는 것을 의미한다.  
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

#### sigmoid 계층

In [17]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out  #역전파시에 사용하기 위해 보관
        
        return out
    
    def backward(self,dout):
        dx = dout * (1.0 - selfout) * self.out  #sigmoid 역전파 간소화 식
        
        return dx

#### affine transformation (아파인 변환) 계층

affine trans. : 신경망의 순전파 시에 수행하는 행렬의 곱 , 보통 노드사이에 스칼라 값이 아닌 넘파이가 흐르기 때문에 많이 사용

In [35]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

# softmax-with-loss 계층

학습시에는 사용, 추론 시에는 배제

*소프트맥스 함수의 손실 함수로 교차 엔트로피 오차를 사용 --> 역전파시에 y-t의 값으로 말끔하게 떨어짐

In [36]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  #손실
        self.y = None  #softmax 출력
        self.t = None  #정답 레이블(원-핫 벡터)
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        
        return self.loss
    
    def backward(self,dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [41]:
import sys, os
sys.path.append(os.pardir)
import numpy as np 
from collections import OrderedDict  #순서가 있는 딕셔너리 클래쓰라고 생각

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

def softmax(a):
    c = np.max(a)  #a배열의 최대값. 오버플로우를 방지하기 위해서 사용한다.
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def cross_entropy_error(y,t):
    if y.ndim == 1:  #y가 1차원이라면, 즉 데이터 하나당 교차 엔트로피를 구하는 것이라면
        t = t.reshape(1,t.size)  #그에 따라 t,y데이터 reshape
        y = y.reshape(1,y.size)
    
    batch_size = y.shape[0]
    
    return -np.sum(t*np.log(y+1e-7)) / batch_size


class TwoLayerNet:
    
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):  #weight_init_std : 정규분포의 스케일 (크기를 맞춰주는 역할인가봄)
        #가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        if t.ndim != 1: 
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W : self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self,x,t):
        #순전파 
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

#### gradient_check : '오차역전파 = 수치 미분'을 확인하는 과정

In [42]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch,t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff) )

W1:9.72079092534882e-05
b1:0.0007604059940871663
W2:0.0023593042151983713
b2:0.06666666678701484
